In [2]:
import os
import random
import math
import numpy as np
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from einops.layers.torch import Rearrange
from einops import repeat

In [3]:
# デバイスの設定
if torch.cuda.is_available():
    DEVICE = "cuda"
elif torch.mps.is_available():
    DEVICE = "mps"
else:
    DEVICE = "cpu"
print(f"Using device: {DEVICE}")

Using device: mps


# GPTモデルの実装

In [ ]:
class SelfAttention(nn.Module):
    """ セルフアテンションを実装 """
    def __init__(self, config, resid_pdrop=0.1, attn_pdrop=0.1, causal=True) -> None:
        """
        Args:
            config: Config
            resid_pdrop : float
                出力projection層のドロップアウト率
            attn_pdrop : float
                Attentionのドロップアウト率
            causal : bool
                causal maskを利用するかどうか判別するフラグ
        """
        super().__init__()
        assert config.n_embd % config.n_head == 0

        # 入力をK, Q, Vにそれぞれ変換する全結合層
        self.key = nn.Linear(config.n_embd, config.n_embd)
        self.query = nn.Linear(config.n_embd, config.n_embd)
        self.value = nn.Linear(config.n_embd, config.n_embd)

        self.attn_drop = nn.Dropout(attn_pdrop)
        self.resid_drop = nn.Dropout(resid_pdrop)

        # Multi-Head Attentionアウトプットの全結合層
        self.proj = nn.Linear(config.n_embd, config.n_embd)

        # torch.trilは行列の右上三角部分をゼロにして返す（予測するトークンの右側をマスク）
        # nn.Moduleのregister_bufferは, モデルのパラメータとならないtensorを追加するのに使われる
        if causal:
            self.register_buffer(
                name="mask",
                tensor=torch.tril(
                    torch.ones(config.block_size, config.block_size)
                ).view(1, 1, config.block_size, config.block_size)
            )
        self.n_head = config.n_head

    def forward(self, x, layer_past=None):
        """
        Args:
            x : torch.Tensor ( b, t, d )
                入力ベクトル系列
                b : バッチサイズ
                t : シークエンス長. コンテクストサイズ (block_size)よりも小さくないといけない
                d : Embedding次元数. 上図のd_model
        Returns:
            y : torch.Tensor ( b, t, d )
        """
        b, t, d = x.size()

        # Key, Que, Valueをそれぞれの全結合層で計算
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        # Multi-Head．d_kやd_vがd_model // n_headsになるような実装だが，必ずしもその必要はない
        k = k.view(b, t, self.n_head, d // self.n_head).transpose(1, 2)
        q = q.view(b, t, self.n_head, d // self.n_head).transpose(1, 2)
        v = v.view(b, t, self.n_head, d // self.n_head).transpose(1, 2)

        # QとKの行列積をとり, sqrt(d_k)でスケール
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))

        # Causal mask
        if hasattr(self, "mask"):
            att = att.masked_fill(self.mask[:, :, :t, :t] == 0, float('-inf'))

        att = F.softmax(att, dim=-1)

        att = self.attn_drop(att)

        # Attention mapとValuesの行列積
        y = att @ v

        # 各headからの出力を結合
        y = y.transpose(1, 2).contiguous().view(b, t, d)

        # Attention出力のprojection層
        y = self.resid_drop(self.proj(y)) # ( b, t, embd_dim )

        return y

## Blockの定義

In [5]:
class Block(nn.Module):
    def __init__(self, config, resid_pdrop=0.1, causal=True) -> None:
        """
        Args:
            config (Config): 設定
            resid_pdrop (float, optional): ドロップアウト確率. Defaults to 0.1.
            causal (bool, optional): 読み取り方向. Defaults to True.
        """
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.attn = SelfAttention(config, causal=causal)

        self.mlp = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(resid_pdrop),
        )

    def forward(self, x):
        """
        Args:
            x: torch.Tensor (b, t, d)
        """
        # LinearNormalizationはAttentionやFeedForwardの前に適用する
        # プラス、残差接続
        x = self.attn(self.ln_1(x)) + x
        x = self.mlp(self.ln_2(x)) + x
        return x